In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Подключаем библиотеки 

# 1. Методы ближайших соседей

1.1 Подключитесь к набору данных. Разберитесь в том, как устроен датасет и какова постановка задачи.

In [ ]:
#Загрузка данных
df = pd.read_csv("/kaggle/input/bank-marketing-dataset/bank.csv")
df.head()

In [ ]:
# Статистика по числовым признакам
df.describe().T

In [ ]:
df.info()

Датасет состоит из данных маркетингового банка(возраст клиента, образование, баланс и тд). Target переменная - deposit. Цель прогнозной модели - проанализировать данные и выяснить внесет ли человек депозит

1.2 Извлеките целевой признак (target). Какая из задач обучения с учителем рассматривается –– классификация или регрессия?

In [ ]:
df['deposit']

Целевой признак (target) - deposit.
    Задача обучения - классификация, так как имеет конечное количество ответов("да" или "нет")

1.3 Каково распределение значений target-переменной? Постройте подходящую визуализацию. Прокомментируйте результат.

In [ ]:
df['deposit'].hist()

Исходя из распределения видно что классы достаточно сбалансированны 

1.4 Проведите необходимую предобработку данных (preprocessing). Для построения моделей с помощью метрических методов все признаки должны быть закодированы числами.

Преобразование категориальных признаков проведем путём кодирования с помощью map()

In [ ]:
new_values = {'yes':  1, 'no': 0} 
df['deposit1'] = df['deposit'].map(new_values)

In [ ]:
df['housing1'] = df['housing'].map(new_values)

In [ ]:
df['loan1'] = df['loan'].map(new_values)

In [ ]:
df['default1'] = df['default'].map(new_values)
df['contact'].value_counts()

In [ ]:
new_values_1 = {'cellular':  1, 'unknown': 0,'telephone' : 1 } 

df['contact1'] = df['contact'].map(new_values_1)
df['contact1'].value_counts()

In [ ]:
new_values_2 = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec':12}
df['month1'] = df['month'].map(new_values_2)

In [ ]:
df = df.drop(['deposit','housing','loan','default','contact', 'month'], axis = 1)

Для признаков имеющих больше двух параметров воспользуемся get_dummies()

In [ ]:
df = pd.get_dummies(df, columns=['marital','poutcome','education'])
df.head().T

1.5 Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split .

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('deposit1', axis=1).drop('job',axis = 1)
y = df['deposit1'] 


# Разделение
# test_size --- доля исходных данных, которую оставляем для валидации
# random_state --- произвольное целое число, для воспроизводимости случайных результатов

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12)
X.info()

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print( X_valid.shape, y_valid.shape)

1.6 Обучите алгоритм классификации KNeighborsClassifier. Оцените качество каждой модели на валидационной выборке с помощью accuracy_score. Сравните результаты и сделайте выводы.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
y_pred = knn.predict(X_valid)

In [ ]:
from sklearn.metrics import accuracy_score

knn.score(X_valid, y_valid) #вычисляем метрику качества

In [ ]:
print('Качество модели:', accuracy_score(y_pred, y_valid)) #другой способ для вычисления метрики

При использовании двух метрик мы получаем идентичный результат, следовательно модель качественная

2. Настройка оптимального числа ближайших 
соседей в методе kNN

2.1 Создайте генератор разбиений, который перемешивает выборку перед созданием блоков ( shuffle=True ). Число блоков n_splits равно 5. Задайте также параметр random_state для воспроизводимости результатов. Например: kf = KFold(n_splits=5, shuffle=True, random_state=42) Найдите показатель качества модели kNN на кросс-валидации. Подумайте, приемлемо ли использование вашей меры (метрики) качества в данной задаче? Прин еобходимости пересчитайте качество с помощью другой метрики из списка.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42) # n_splits играет роль K
scores = cross_val_score(knn, X, y, cv=kf, scoring='accuracy')
print('Массив значений метрики:', scores)

Приемлемо использовать данную метрику, так как качество модели от изменений метрики меняется не значительно

2.2 Осуществите кросс-валидацию модели при числе соседей k ∈ [1;50]. Используйте GridSearchCV . При каком k качество получилось наилучшим? Чему равна эта оценка качества? Постройте график значений метрики в зависимости от k ( matplotlib.pyplot.plot() ).

In [ ]:
from sklearn.model_selection import GridSearchCV

knn_params={'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn,knn_params, scoring = 'accuracy', cv=kf)
knn_grid.fit(X_train, y_train)

In [ ]:
knn_grid.best_estimator_ # Лучшая модель

In [ ]:
knn_grid.best_params_ # Лучшие значения параметров

In [ ]:
knn_grid.best_score_

In [ ]:
# Результаты кросс-валидации в виде таблицы
pd.DataFrame(knn_grid.cv_results_).T

In [ ]:
# Валидационная кривая
import matplotlib.pyplot as plt

results_df = pd.DataFrame(knn_grid.cv_results_)
plt.plot(results_df['param_n_neighbors'], results_df['mean_test_score'])

plt.xlabel('Number of neighbors')
plt.ylabel('Test accuracy')
plt.title('Validation curve')
plt.show()

3. Выбор метрики в методе kNN 

3.1 Переберите разные варианты значений параметра p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (удобно использовать функцию numpy.linspace ). Используйте KNeighborsClassifier или KNeighborsRegressor с оптимальным значением n_neighbors , найденным ранее. Задайте опцию weights='distance' –– данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества снова используйте accuracy . Качество оценивайте с помощью кросс-валидации по 5 блокам.

In [ ]:
p_params = {'p': np.linspace(1,10,200)}
knn = KNeighborsClassifier(n_neighbors=27, weights = 'distance', n_jobs = -1)
cv = GridSearchCV(knn, p_params, cv = kf, scoring='accuracy', verbose = 100)
cv.fit(X,y)

In [ ]:

cv_result = pd.DataFrame(cv.cv_results_)
cv_result


In [ ]:
cv.best_estimator_

In [ ]:
cv.best_score_

3.2 Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей.

In [ ]:
cv.best_params_

При p = 1.0452261306532664 качество на кросс-валидации оказалось оптимальным

4. Другие метрические методы

Поэкспериментируйте с другими метрическими методами для задач регрессии и классификации, представленными в библиотеке Scikit-learn: 
* RadiusNeighborsClassifier ; 
* RadiusNeighborsRegressor ; 
* NearestCentroid . 


Рассмотрим метод RadiusNeighborsClassifier

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
def predy(r):
    neigh = RadiusNeighborsClassifier(radius=r)
    neigh.fit(X, y)
    y_pred = neigh.predict(X_valid)
    return y_pred

accuracy_score(predy(1), y_valid)

Результат выглядит подозрительно хорошо, посмотрим результат на большей выборке радиусов

In [ ]:
t = np.arange(1,100)
ac = [accuracy_score(predy(ts),y_valid) for ts in t  ]
plt.plot(t,ac)

Исходя из этого, в данной задаче эта модель - достаточно эффективна

Рассмотрим также метод NearestCentroid

In [ ]:
from sklearn.neighbors import NearestCentroid
nc = NearestCentroid()
nc.fit(X_train, y_train)
NearestCentroid()
y_pred_nc = nc.predict(X_valid)
accuracy_score(y_valid, y_pred_nc)

Результат работы этого метода хуже чем в методе kNN. В данной задаче эта модель неэффективна